## User Satisfaction Analysis

#### Import Modules

In [1]:
import pickle
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys, os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from plot_dataframe import PlotDataFrame
from utils import Util

#### Read Data